## Programmation concurente - SUJET

**Ressources** : <a href="https://webge.fr/dokuwiki/doku.php?id=python:accueilpython" target="_blank"><button>Wiki Python sur WebGE</button></a> <a href="https://realpython.com/python-logging/" target="_blank"><button>Logging in Python</button></a> <a href="https://realpython.com/intro-to-python-threading/" target="_blank"><button>An Intro to Threading in Python</button></a>

### Sommaire
<ol>
    <li>Introduction
        <ol>
            <li>Les processus</li>
            <li>Les threads</li>
            <li>Quelle est la différence entre un Thread et un Processus ?</li>
        </ol>
    </li>
    <li>Illustration de l'ordre d'exécution de threads</li>
    <li>Ressource partagée</li>
        <ol>
            <li>Problème de concurrence</li>
            <li>Section critique</li>
        </ol>
    <li>Interblocage</li>
    <li>Synthèse</li>
    <li>A savoir</li>
</ol>

### 1. Introduction
#### 1.a Les processus
<em>Grâce à leur système d'exploitation **multitâche**, les ordinateurs exécutent plusieurs programmes de façon **concurrente**. L'exécution d'un programme s'appelle un **processus**. C'est le système d'exploitation, et en particulier l'**ordonnanceur** (une des fonctionnalités du **noyau**), qui détermine quel processus s'exécute à un instant donné. Le fait pour un processus d'être interrompu s'appelle une **commutation de contexte**. Plusieurs processus s'exécutant de façon concurrente peuvent s'**interbloquer** s'ils attendent de pouvoir accéder à un même ensemble de **ressources en accès exclusif**. </em><br>
<img src="img/se.png"> <br>
#### 1.b Les threads
<em>Les **threads** ou processus légers sont des "sous-processus", démarrés par un processus et s'exécutant de manière concurrente avec le reste du programme. L'accès à des ressources par plusieurs threads peut être protégé par des **verrous**. Une portion de code comprise entre l'acquisition et le relâchement d'un verrou s'appelle une **section critique**. <br>
Le module threading de la bibliothèque standard Python permet de démarrer des threads.</em><br>
#### 1.c Quelle est la différence entre un Thread et un Processus ?
<em>Les threads (du même processus) s'exécutent dans un espace mémoire partagé, tandis que les processus s'exécutent dans des espaces mémoire différents.</em>
<img src="img/threads.png"> 

-----

### Objectifs
> **Illustrer l'ordre d'exécution de threads, les problèmes de concurrence et d'interblocage**. 

-----

### 2. Illustration de l'ordre d'exécution de threads
<em>Dans le code ci-dessous, le programme principal crée quatre threads <span class="code"><strong>th</strong></span> à l'aide de l'instruction <span style="font-family:Consolas;font-weight:bold;font-style:normal">threading.Thread(target=hello, arg=[n])</span>. Lorsque l'on crée un thread, on lui transmet une fonction et la liste des arguments de cette fonction. La méthode **start()** lance l'exécution du thread en tâche de fond. Cette méthode rend la main et le programme principal continue de s'exécuter de façon concurrente au(x) thread(s) démarré(s). Une fois la boucle <strong>for</strong> exécutée, le programme comporte cinq *threads* : les quatre démarrés par start() plus celui associé au programme principal. Un compteur <strong>cmpt</strong> est créé dans chaque thread pour illustrer leur ordre d'exécution.</em>

<strong>Note</strong> : la bibliothèque <a href="https://docs.python.org/fr/3/howto/logging.html" target="_blank">logging</a> est dédiée à la journalisation.

In [ ]:
# Programmation concurente - Illustration de l'ordre d'exécution de threads 
import threading
import logging # Cette bibliothèque est dédiée à la journalisation
import time

# Fonction associée aux threads 0 à 3
def hello(num):
    logging.info(f"Thread {num}: démarrage")
    for i in range(5):
        logging.info(f"Thread {num} : cmpt{num}={i}")
        time.sleep(0.5) # Simulation d'un programme plus long
    logging.info(f"Thread {num}: terminé")

# Programme principal
# Formatage des informations affichées lors du déroulement du programme
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO, # filename='thread.log', filemode='a',
                        datefmt="%H:%M:%S", encoding='utf-8')
for numth in range(4):                                # Création des threads 0 à 3
    th = threading.Thread(target=hello, args=[numth]) # l'argument de type target est une fonction et l'argument 
                                                      # args est un tableau d'arguments passés à la fonction.
                                                      # Ici, on passe le numéro numth du thread th à des fins d'affichage.
    logging.info(f"PPrinc : avant de lancer le Thread {numth}")
    th.start()                                                                                          

> **Activité 1 <br>
> Exécutez** plusieurs fois le code ci-dessus. Que peut-on dire de l'ordre d'exécution des threads et de l'ordre dans lequel ils s'arrêtent ?

<p style="color:blue; font-weight:bold"><em>Réponse Activité 1</em></p>
A rédiger ici

<table><tr><td style="color:red; font-size:14px"><strong>REMARQUE</strong> : l'ordre dans lequel sont démarrés les threads ne donne aucune indication sur l'ordre dans lequel ils peuvent se terminer.</td></tr></table>

#### 3. Ressource partagée
#### 3.A Problème de concurence
<em>Dans le programme ci-dessous, la variable globale **COMPTEUR** représente une **ressource partagée** par plusieurs threads. Comme hello dans le programme précédent, la fonction **incrc** s'exécute dans des threads.</em>

In [1]:
# Programmation concurente - Compteur partagé
# Illustration du problème de concurrence v1
import threading
import logging
import time

COMPTEUR = 0 # Ressource partagée

# Fonction associée aux threads 0 à 3
def incrc(n):
    global COMPTEUR
    for _ in range(10):
        v = COMPTEUR
        logging.info(f"Thread {n} - cpt={COMPTEUR}")
        COMPTEUR = v + 1

# Programme principal
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S", encoding='utf-8')
th=[] # tableau de threads
for n in range(4):
    t = threading.Thread(target=incrc, args=[n])
    t.start()
    th.append(t)

for t in th: # Permet d'attendre que tous les threads soient terminés avant de poursuivre
    t.join() # dans le programme principal

logging.info(f"Valeur finale = {COMPTEUR}") # Cette ligne est exécutée lorsque tous les threads sont terminés

14:49:07: Thread 0 - cpt=0
14:49:07: Thread 1 - cpt=0
14:49:07: Thread 2 - cpt=0
14:49:07: Thread 0 - cpt=1
14:49:07: Thread 3 - cpt=1
14:49:07: Thread 1 - cpt=1
14:49:07: Thread 2 - cpt=1
14:49:07: Thread 0 - cpt=2
14:49:07: Thread 3 - cpt=2
14:49:07: Thread 1 - cpt=2
14:49:07: Thread 2 - cpt=2
14:49:07: Thread 0 - cpt=3
14:49:07: Thread 3 - cpt=3
14:49:07: Thread 1 - cpt=3
14:49:07: Thread 2 - cpt=3
14:49:07: Thread 0 - cpt=4
14:49:07: Thread 3 - cpt=4
14:49:07: Thread 1 - cpt=4
14:49:07: Thread 2 - cpt=4
14:49:07: Thread 0 - cpt=5
14:49:07: Thread 3 - cpt=5
14:49:07: Thread 1 - cpt=5
14:49:07: Thread 2 - cpt=5
14:49:07: Thread 0 - cpt=6
14:49:07: Thread 3 - cpt=6
14:49:07: Thread 1 - cpt=6
14:49:07: Thread 2 - cpt=6
14:49:07: Thread 0 - cpt=7
14:49:07: Thread 3 - cpt=7
14:49:07: Thread 1 - cpt=7
14:49:07: Thread 2 - cpt=7
14:49:07: Thread 0 - cpt=8
14:49:07: Thread 3 - cpt=8
14:49:07: Thread 1 - cpt=8
14:49:07: Thread 2 - cpt=8
14:49:07: Thread 0 - cpt=9
14:49:07: Thread 3 - cpt=9
1

> **Activité 2. Analyse et tests** du programme ci-dessus. <br>
> 1. Que fait la fonction incrc ? 
> 2. Quelle doit être la valeur de COMPTEUR à la fin du programme ? 
> 3. Testez le programme plusieurs fois. La valeur est-elle toujours celle supposée ? Pourquoi ?

<p style="color:blue; font-weight:bold"><em>Réponse Activité 2</em></p>
A rédiger ici

#### 3.B Section critique
<em>Pour corriger le problème identifié dans le code précédent, il faut rendre <strong>EXCLUSIF</strong> l'accès à la variable <strong>COMPTEUR</strong>. On peut pour cela utiliser un verrou. Un <strong>verrou</strong> est un objet que l'on essaye d'acquérir. Si un thread est le premier à en faire la demande, il l'acquiert. Il peut le rendre à tout moment. Si en revanche un autre thread le détient alors tous les threads qui tentent d'y accéder sont bloqués jusqu'à ce qu'il soit libéré. On construit un verrou avec la méthode <strong>Lock()</strong> du <strong>module threading</strong>. On peut alors tenter de l'acquérir avec la méthode <strong>acquire()</strong> et le rendre avec la méthode <strong>release()</strong>.</em>

<table><tr><td style="color:blue; font-size:14px"><strong>NOTE</strong> : Une portion de code protégée par un verrou s'appelle une <strong>SECTION CRITIQUE</strong>.</td></tr></table>

In [ ]:
# Programmation concurente - Compteur partagé
# Illustration du problème de concurrence v1
import threading
import logging
import time

COMPTEUR = 0 # Ressource partagée
verrou = threading.Lock() # construction du verrou

# Fonction associée aux threads 0 à 3
def incrc(n):
    global COMPTEUR
    for _ in range(10):
        ...
        v = COMPTEUR
        logging.info(f"Thread {n} - cpt={COMPTEUR}")
        COMPTEUR = v + 1
        ...

# Programme principal
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S", encoding='utf-8')
th=[] # tableau de threads
for n in range(4):
    t = threading.Thread(target=incrc, args=[n])
    t.start()
    th.append(t)

for t in th: # Permet d'attendre que tous les threads soient terminés avant de poursuivre
    t.join() # dans le programme principal

logging.info(f"Valeur finale = {COMPTEUR}") # Cette ligne est exécutée lorsque tous les threads sont terminés

> **Activité 3**<br>
> Un verrou est créé dans le programme ci-dessus par : <span style="font-family:Consolas;font-weight:bold;font-style:normal">verrou = threading.Lock()</span> <br>
> L'objet verrou possède deux méthodes : **acquire()** et **release()**  <br>
> <br>
> a) Placez le verrou dans le code ci-dessus pour protéger la section critique. <br>
> b) Testez le programme avec différentes bornes pour la boucle for. Que remarquez-vous ? <br>
> c) Expliquez pourquoi on a corrigé le problème de concurrence entre les threads t0, t1, t2 et t3.

<p style="color:blue; font-weight:bold"><em>Réponse Activité 3</em></p>
A rédiger ici

### 4. Interblocage
L'interblocage se produit lorsque des processus concurrents **s'attendent mutuellement**. L'utilisation de plusieurs verrous rend le risque d'**interblocages** possible.<br>
Dans l'exemple ci-dessous **P1** et **P2** sont **interbloqués** car : <br>
- Le processus **P1** dispose de la ressource **D1** et attend la ressource **D2**. <br>
- Le processus **P2** dispose de la ressource **D2** et attend la ressource **D1**. <br>
<img src="img/interblocage.png">

In [4]:
# Illustration de linterblocage
# La fonction P1 essaye d'acquérir d'abord verrou1 puis verrou2, alors que P2 essaye de 
# les acquérir dans l'ordre inverse.
# Si on exécute ce programme, il a de grandes chances de se retrouver bloqué.
import threading
import logging

verrou1 = threading.Lock()
verrou2 = threading.Lock()

def p1():
    verrou1.acquire()
    logging.info("P1 a acquit D1")  
    verrou2.acquire()
    logging.info("P1 a acquit D2")
    verrou2.release()
    logging.info("P1 a rendu D2")
    verrou1.release()
    logging.info("P1 a rendu D1")
    
    
def p2():
    verrou2.acquire()
    logging.info("P2 a acquit D2")  
    verrou1.acquire()
    logging.info("P2 a acquit D1")
    verrou1.release()
    logging.info("P2 a rendu D1")
    verrou2.release()
    logging.info("P2 a rendu D2")

# Programme principal
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S", encoding='utf-8')

t1 = threading.Thread(target=p1, args=[])
t2 = threading.Thread(target=p2, args=[])
t1.start()
t2.start()


14:51:32: P1 a acquit D1
14:51:32: P2 a acquit D2


> **Activité 4a. Analyse** du programme ci-dessus <br>
Quel pourrait être le texte affiché par le programme : <br>
a) S'il ne se bloque pas ? <br>
b) S'il se bloque ?  

<p style="color:blue; font-weight:bold"><em>Réponse Activité 4a</em></p>
A rédiger ici

> **Activité 4b** <br>
>**Supprimer** l'interblocage dans le programme ci-dessous.

In [ ]:
# Illustration de linterblocage
# La fonction P1 essaye d'acquérir d'abord verrou1 puis verrou2, alors que P2 essaye de 
# les acquérir dans l'ordre inverse.
# Si on exécute ce programme, il a de grandes chances de se retrouver bloqué.
import threading
import logging

verrou1 = threading.Lock()
verrou2 = threading.Lock()

def p1():
    verrou1.acquire()
    logging.info("P1 a acquit D1")  
    verrou2.acquire()
    logging.info("P1 a acquit D2")
    verrou2.release()
    logging.info("P1 a rendu D2")
    verrou1.release()
    logging.info("P1 a rendu D1")
    
    
def p2():
    verrou2.acquire()
    logging.info("P2 a acquit D2")  
    verrou1.acquire()
    logging.info("P2 a acquit D1")
    verrou1.release()
    logging.info("P2 a rendu D1")
    verrou2.release()
    logging.info("P2 a rendu D2")

# Programme principal
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S", encoding='utf-8')

t1 = threading.Thread(target=p1, args=[])
t2 = threading.Thread(target=p2, args=[])
t1.start()
t2.start()


### 5. SYNTHESE

> **Activité 5**<br>
> En vous inspirant du programme du paragraphe 2, **écrivez** un programme qui crée et démarre **quatre** fonctions concurrentes affichant plusieurs fois un message de bienvenue personnalisé (maximum **dix** fois le message).<br>

> *Exemple de résultats attendus*<br>
Bonjour, je suis le thread 0 et ceci est le message 1<br>
... <br>
Message de bienvenue du thread 1 qui transmet son message 3<br>
... <br>

In [ ]:
# Programmation concurente - Messages différents dans chaque thread

import threading
import logging

'''
numth : numéro du thread
msg : tableau contenant les messages à afficher
nb : tableau contenant le nombre de messages que chaque thread doit produire
'''

# A compléter

> **Activité 6a** <br>
> On considère un petit système embarqué : un **microcontrôleur** relié à trois **LED A, B, C**. Une LED peut être éteinte ou éclairée et on peut configurer sa couleur. On dispose de trois programmes qui affichent des signaux lumineux en faisant clignoter les LED. Chaque programme possède une LED primaire et une LED secondaire. <br>
> - Le programme P1 émet ses signaux sur A (primaire) et B (secondaire) en vert.<br>
> - Le programme P2 émet ses signaux sur B (primaire) et C (secondaire) en bleu.<br>
> - Le programme P3 émet ses signaux sur C (primaire) et A (secondaire) en rouge.<br>
>
> Comme les LED ne peuvent pas être configurées dans deux couleurs en même temps, le système propose deux primitives **acquerirLED(nom)** et **rendreLED(nom)** qui permettent respectivement d'acquérir et de relâcher une LED. <br> **nom** prend la valeur primaire ou secondaire. <br> Si une LED est déjà acquise par un programme Pi alors acquerirLED(nom) dans un programme Pj bloque Pj (i différent de j).<br>
> On suppose que chacun des trois programmes P1, P2, P3 effectue les **actions** suivantes en boucle :<br>
> 1. acquérir la LED primaire
> 2. acquérir la LED secondaire
> 3. configurer les couleurs
> 4. émettre des signaux
> 5. rendre la LED secondaire 
> 6. rendre la LED primaire puis
> recommencer en 1
>
> **Montrer** qu'il existe un entrelacement des exécutions qui place **P1, P2 et P3 en interblocage**.

<p style="color:blue; font-weight:bold"><em>Réponse 6a</em></p>
A rédiger ici

> **Activité 6b** <br>
Téléchargez, copiez et complétez le code situé ici : https://gist.github.com/WebGE/f24c17bb13f10b38eaf21725451e3754
>
> *Exemple de résultats attendus*<br>
11:21:14: LedA=vert par P1 <br>
11:21:14: LedB=vert par P1 <br>
11:21:14: LedC=rouge par P3 <br>
11:21:14: LedB relachée par P1 <br>
11:21:14: LedB=bleu par P2 <br>
11:21:14: LedA relachée par P1 <br>
11:21:14: LedA=rouge par P3 <br>
11:21:14: LedA relachée par P3 <br>
11:21:14: LedA=vert par P1 <br>
11:21:14: LedC relachée par P3 <br>
11:21:14: LedC=bleu par P2 <br>
11:21:14: LedC relachée par P2 <br>

In [ ]:
# MICROLED - Illustration de l'interblocage dans la commande des Leds
# A compléter

### 6. A retenir
<em>Les systèmes d'exploitation multitâches sont la norme. Ils permettent d'exécuter de façon <strong>concurrente</strong> plusieurs programmes. L'exécution d'un programme s'appelle un <strong>processus</strong>. C'est le système d'exploitation et en particulier l'<strong>ordonnanceur</strong>, qui détermine quel processus s'exécute à un instant donné. Le fait pour un processus d'être interrompu s'appelle une <strong>commutation de contexte</strong>. Plusieurs processus s'exécutant de façon concurrente peuvent s'<strong>interbloquer</strong> s'ils attendent de pouvoir accéder à un même ensemble de <strong>ressources en accès exclusif</strong>. Les <strong>threads</strong> ou <strong>processus légers</strong> sont des "sous-processus" s'exécutant de manière concurrente. L'accès à des ressources par plusieurs threads peut être protégé par des <strong>verrous</strong>. Une portion de code comprise entre l'acquisition et le relâchement d'un verrou s'appelle une <strong>section critique</strong>.</em> Numérique et Sciences Informatiques - <span class="codepy">ellipses</span>